In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
from sklearn import datasets
from google.cloud import storage

In [13]:
#INPUT_PATH = "D:\\Northeastern\\100 pct undersample split\\"
INPUT_PATH = "gs://kkbox-data/data_100_pct_undersample/"
split = "val"

#X = pd.read_csv(INPUT_PATH + f"X_{split}.csv")
X = pd.read_csv(INPUT_PATH + "X_{}.csv".format(split))
X.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,I+xrcRaaEC4c5hEdiBZDfcyA3zmPFdE3Q9T9YMnAXmI=,12,23,male,9,20150117
1,VbgWtekNTFyscLPhCw0Qp3aFwWq8BNYHyhVzEzwZArM=,5,0,NaN,9,20140920
2,03sGSwdjTYeZJourLdovEeJBxylYOzcghPkNIv4+TFI=,15,51,female,9,20110622
3,3zDR0Bg5aQmJShlZmcqxllw6r4JfVIYbvjYIxcWArHQ=,13,0,NaN,7,20120513
4,JBcWjLEPgUG2sfZ8UTpi9LuwsKHVjWhe32YLmd5dg6Q=,1,0,NaN,7,20130705


# Gender
We treat 'unknown' as a 3rd category of gender with the assumption being that the user's decision to not provide gender may be relevant to our predictions

In [14]:
X.loc[X["gender"] == "male","gender"] = "M"
X.loc[X["gender"] == "female","gender"] = "F"
X.loc[:,"gender"]=X["gender"].fillna("U")

enc_gender = preprocessing.OneHotEncoder(drop="first",categories="auto")
gender_onehot = enc_gender.fit_transform(X.gender.values.reshape(-1,1)).toarray()
# Drop old gender column and merge new onehot encoded columns
X=X.drop(["gender"],axis=1)
X["gender_male"] = gender_onehot[:,0]
X["gender_unknown"] = gender_onehot[:,1]
X.head()

,msno,city,bd,registered_via,registration_init_time,gender_male,gender_unknown
0,I+xrcRaaEC4c5hEdiBZDfcyA3zmPFdE3Q9T9YMnAXmI=,12,23,9,20150117,1.0,0.0
1,VbgWtekNTFyscLPhCw0Qp3aFwWq8BNYHyhVzEzwZArM=,5,0,9,20140920,0.0,1.0
2,03sGSwdjTYeZJourLdovEeJBxylYOzcghPkNIv4+TFI=,15,51,9,20110622,0.0,0.0
3,3zDR0Bg5aQmJShlZmcqxllw6r4JfVIYbvjYIxcWArHQ=,13,0,7,20120513,0.0,1.0
4,JBcWjLEPgUG2sfZ8UTpi9LuwsKHVjWhe32YLmd5dg6Q=,1,0,7,20130705,0.0,1.0


# Registration Date
We convert registration date into an integer containing the number of days since registration. Those values are then standardized.

In [15]:
reg_date = pd.to_datetime(X.registration_init_time, format="%Y%m%d")
# Assuming here that our training date for predicting February 2017 is Jan 1, 2017
train_date = pd.to_datetime("20170131", format="%Y%m%d")
reg_days = (train_date - reg_date).dt.days

reg_days_scaled = preprocessing.scale(reg_days)
X["reg_days_scaled"] = pd.Series(reg_days_scaled)
X=X.drop(["registration_init_time"], axis=1)
X.head()

,msno,city,bd,registered_via,gender_male,gender_unknown,reg_days_scaled
0,I+xrcRaaEC4c5hEdiBZDfcyA3zmPFdE3Q9T9YMnAXmI=,12,23,9,1.0,0.0,-0.766996
1,VbgWtekNTFyscLPhCw0Qp3aFwWq8BNYHyhVzEzwZArM=,5,0,9,0.0,1.0,-0.654560
2,03sGSwdjTYeZJourLdovEeJBxylYOzcghPkNIv4+TFI=,15,51,9,0.0,0.0,0.466018
3,3zDR0Bg5aQmJShlZmcqxllw6r4JfVIYbvjYIxcWArHQ=,13,0,7,0.0,1.0,0.158001
4,JBcWjLEPgUG2sfZ8UTpi9LuwsKHVjWhe32YLmd5dg6Q=,1,0,7,0.0,1.0,-0.236941


# Registration Method
We one-hot encode the registration methods

In [16]:
enc_registration = preprocessing.OneHotEncoder(drop="first",categories="auto")
registration_onehot = enc_registration.fit_transform(X.registered_via.values.reshape(-1,1)).toarray()
registration_df = pd.DataFrame(registration_onehot)
registration_df.columns = enc_registration.get_feature_names(["registered_via"])
X = pd.concat([X,registration_df], axis=1)
X = X.drop(["registered_via"],axis=1)
X.head()

,msno,city,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9
0,I+xrcRaaEC4c5hEdiBZDfcyA3zmPFdE3Q9T9YMnAXmI=,12,23,1.0,0.0,-0.766996,0.0,0.0,1.0
1,VbgWtekNTFyscLPhCw0Qp3aFwWq8BNYHyhVzEzwZArM=,5,0,0.0,1.0,-0.654560,0.0,0.0,1.0
2,03sGSwdjTYeZJourLdovEeJBxylYOzcghPkNIv4+TFI=,15,51,0.0,0.0,0.466018,0.0,0.0,1.0
3,3zDR0Bg5aQmJShlZmcqxllw6r4JfVIYbvjYIxcWArHQ=,13,0,0.0,1.0,0.158001,0.0,1.0,0.0
4,JBcWjLEPgUG2sfZ8UTpi9LuwsKHVjWhe32YLmd5dg6Q=,1,0,0.0,1.0,-0.236941,0.0,1.0,0.0


# City
We one-hot encode the city

In [17]:
enc_city = preprocessing.OneHotEncoder(drop="first",categories="auto")
city_onehot = enc_city.fit_transform(X.city.values.reshape(-1,1)).toarray()
city_df = pd.DataFrame(city_onehot)
city_df.columns = enc_city.get_feature_names(["city"])
X = pd.concat([X,city_df], axis=1)
X = X.drop(["city"],axis=1)
X.head()

,msno,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,city_4,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,I+xrcRaaEC4c5hEdiBZDfcyA3zmPFdE3Q9T9YMnAXmI=,23,1.0,0.0,-0.766996,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,VbgWtekNTFyscLPhCw0Qp3aFwWq8BNYHyhVzEzwZArM=,0,0.0,1.0,-0.654560,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,03sGSwdjTYeZJourLdovEeJBxylYOzcghPkNIv4+TFI=,51,0.0,0.0,0.466018,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3zDR0Bg5aQmJShlZmcqxllw6r4JfVIYbvjYIxcWArHQ=,0,0.0,1.0,0.158001,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JBcWjLEPgUG2sfZ8UTpi9LuwsKHVjWhe32YLmd5dg6Q=,0,0.0,1.0,-0.236941,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Birthday

Scaling the BD values. Others have indicated that this feature is predictive (despite the strange, negative values)

In [18]:
X["bd"] = preprocessing.scale(X.bd)
X.head()

,msno,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,city_4,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,I+xrcRaaEC4c5hEdiBZDfcyA3zmPFdE3Q9T9YMnAXmI=,0.294421,1.0,0.0,-0.766996,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,VbgWtekNTFyscLPhCw0Qp3aFwWq8BNYHyhVzEzwZArM=,-1.139768,0.0,1.0,-0.654560,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,03sGSwdjTYeZJourLdovEeJBxylYOzcghPkNIv4+TFI=,2.040390,0.0,0.0,0.466018,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3zDR0Bg5aQmJShlZmcqxllw6r4JfVIYbvjYIxcWArHQ=,-1.139768,0.0,1.0,0.158001,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JBcWjLEPgUG2sfZ8UTpi9LuwsKHVjWhe32YLmd5dg6Q=,-1.139768,0.0,1.0,-0.236941,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X.to_csv(INPUT_PATH + "X_{}_transformed.csv".format(split),index=False)